In [1]:
#############################################
##  Node-level Resource Untilization Plot
##     (for every VTK ouput step)
#############################################
# Note: "Nodename per rank" are listed in a single line following every VTK output line

import numpy as np
import linecache
#%matplotlib inline # suppress interactive display by default
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [2]:
###################################
##### PROVIDE INPUTS HERE !!! #####
###################################

# Specify the input file to read
filename = "/media/data/emily/workspace/esamoa/console.out"

# Specify the output figure path & name
figname_prefix = '/media/data/emily/Desktop/testfig/impi_res_'
figname_suffix = '.pdf'

# Specify application number (this determines the color used the application)
appid = 1

# Setup the Nodes visual grid size: nx-by-ny
nx,ny = (8,4)
# Specify CPUs per node: cx-by-cy
cx,cy = (4,4)

# Prodive a "node_name" to "node_id" mapping
node_name_id_map = dict()

nid = 0
for i in range(8):
    node_name = "node"+str(i)
    node_name_id_map[node_name] = nid
    nid += 1
    
### For testing only
#for n in node_name_id_map:
#    print(n, node_name_id_map[n])

In [3]:
###########################
##### Data Processing #####
###########################

# Function for removing duplicate nodes
# Return a list containing only unique nodes
def get_unique(nodes):
    unique_nodes = []
    for n in nodes:
        if n not in unique_nodes:
            unique_nodes += [n]
    return unique_nodes

# Get unique node lists for every OUTPUT step from file
node_lists = []
f = open(filename, 'r')
for i, line in enumerate(f,1):
    # find the "xxx OUTPUT: xxx" line
    if "OUTPUT:" in line:
        # the next line is the "nodename per rank"
        nodes = linecache.getline(filename, i+1).split()
        # --------- TODO: remove this later -------------
        if (nodes[0] == '\x1b[97m(r0,t0)'):
            continue
        #------------------------------------------------
        # Reduce to unique nodes
        unique_nodes = get_unique(nodes)
        # append to node lists
        node_lists += [unique_nodes]
f.close()

### For testing only
# for l in node_lists:
#     print(l)

In [4]:
################################
##### Plotting Definitions #####
################################

# define color code
colorcode = [
        'grey',      # CPU is idle or in transition state
        'lightpink', # CPU is assigned to App1
        'skyblue',   # CPU is assigned to App2
        'khaki',     # CPU is assigned to App3
        'lightgreen' # CPU is assigned to App4
]

# Node patch config
node_size_x = 2.0
node_size_y = 2.0
node_patch_size_x = node_size_x * 0.9
node_patch_size_y = node_size_y * 0.9
node_patch_offset_x = (node_size_x - node_patch_size_x) * 0.5
node_patch_offset_y = (node_size_y - node_patch_size_y) * 0.5

# CPU patch config
cpu_size_x = node_patch_size_x / float(cx)
cpu_size_y = node_patch_size_y / float(cy)
cpu_patch_size_x = cpu_size_x * 0.9
cpu_patch_size_y = cpu_size_y * 0.9
cpu_patch_offset_x = (cpu_size_x - cpu_patch_size_x) * 0.5
cpu_patch_offset_y = (cpu_size_y - cpu_patch_size_y) * 0.5

# Function to convert a list of node_name to a list of node_id
def get_node_ids(node_names):
    ids = []
    for n in node_names:
        ids += [node_name_id_map[n]]
    return ids

# Function to get node color
def get_node_color(n, nodes_in_use):
    nodeids_in_use = get_node_ids(nodes_in_use)
    return colorcode[appid] if (n in nodeids_in_use) else colorcode[0]

# Get the node patch position (x,y)
def getxy(node_id):
    y = ny - 1 - int(node_id / nx)
    x = int(node_id % nx)
    return (x * node_size_x + node_patch_offset_x, \
            y * node_size_y + node_patch_offset_y)

# Get the cpu patch position
def getxy_cpu(node_id, cpu_x, cpu_y):
    (x,y) = getxy(node_id)
    return (x + cpu_x*cpu_size_x + cpu_patch_offset_x, \
            y + cpu_y*cpu_size_y + cpu_patch_offset_y)

# add cpus patches for a node to...
def add_cpus_patches(cpus, node_id, nodes_in_use):
    cpu_color = get_node_color(node_id, nodes_in_use)
    # Add list of cpu patches for the node
    for j in range(cy):
        for i in range(cx):      
            cpus[node_id*(cx*cy) + (j*cx+i)] = patches.Rectangle(
                    getxy_cpu(node_id, i,j),
                    cpu_patch_size_x,
                    cpu_patch_size_y,
                    facecolor=cpu_color)

# Function for drawing 1 frame
def draw_frame(frame_id, nodes_in_use):
    # Figure setup
    fig = plt.figure()
    ax = fig.add_subplot(111, aspect='equal')
    ax.set_xlim([0,nx*node_size_x])
    ax.set_ylim([0,ny*node_size_y])
    ax.axis('off')
    ax.set_frame_on(False)
    # Create a list cpu patches (cx*cy patches per node) for drawing
    cpus = dict()
    for n in range(nx*ny):
        add_cpus_patches(cpus, n, nodes_in_use)
    # Draw the node patches
    for c in cpus:
        ax.add_artist(cpus[c]) 
    # Write node number to each node (fancy)
    for n in range(nx*ny):
        nodepatch = patches.Rectangle(getxy(n), node_patch_size_x, node_patch_size_y)
        x,y = nodepatch.get_xy()
        center_x = x + nodepatch.get_width()/2.0
        center_y = y + nodepatch.get_height()/2.0
        # Put node number at the center of the patch
        ax.annotate(n, (center_x, center_y), \
                    color='black', weight='normal', fontsize=12, ha='center', va='center')    
    # Save and close figure
    filename = figname_prefix + str(frame_id) + figname_suffix
    plt.savefig(filename)
    plt.close(fig)


In [5]:
# Test plotting for 1 frame
frame_id = 15
l = node_lists[frame_id]
print(l)
draw_frame(frame_id, l)

['node0', 'node1', 'node2', 'node3', 'node4', 'node5']


In [ ]:
# Draw all frames
for i in range(len(node_lists)):
    draw_frame(i,node_lists[i])
    